# Import from ELN

Importing data from an electronic laboratory notebook

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from aiida import load_profile

load_profile();

In [ ]:
from aiidalab_widgets_base import AiidaNodeViewWidget, OpenAiidaNodeInAppWidget, ElnImportWidget
import urllib.parse as urlparse
from aiidalab_widgets_base import viewer
from traitlets import dlink

from jsonschema import validate, RefResolver, Draft7Validator, ValidationError
import json
from pyld import jsonld

In [ ]:
def load_json(filepath: str) -> dict:
    return json.load(open(filepath, "r"))

def find_refs(obj, refs=None) -> list[str]:
    if refs is None:
        refs = []
        
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == '$ref':
                refs.append(value[2:])
            else:
                find_refs(value, refs)
    
    return refs

def find_ref_schemas(schemas_filenames: list[str], refs = None):
    if refs is None:
        refs = []
        
    for schema_filename in schemas_filenames:
        schema = load_json(f"/home/jovyan/aiida-openbis/Notebooks/New_Metadata_Schemas/{schema_filename}")
        ref_schemas_filenames = find_refs(schema)
        
        if schema_filename not in refs:
            refs.append(schema_filename)
            find_ref_schemas(ref_schemas_filenames, refs)
    
    return refs

def set_up_validator(json_schema_filename):
    schemas_filenames = [json_schema_filename]
    all_schemas_filenames = find_ref_schemas(schemas_filenames)

    first_schema = load_json(f"/home/jovyan/aiida-openbis/Notebooks/New_Metadata_Schemas/{all_schemas_filenames[0]}")
    resolver = RefResolver(base_uri="http://example.com/", referrer = first_schema)

    for referenced_schema_filename in all_schemas_filenames:
        referenced_schema = load_json(f"/home/jovyan/aiida-openbis/Notebooks/New_Metadata_Schemas/{referenced_schema_filename}")
        resolver.store[referenced_schema["$id"]] = referenced_schema

    validator = Draft7Validator(first_schema, resolver=resolver)
    
    return validator

# Function to expand context terms
def expand_context(base_context):
    expanded_context = {}
    for key, value in base_context.items():
        if isinstance(value, str) and ':' in value:
            prefix, suffix = value.split(':', 1)
            if prefix in base_context:
                expanded_context[key] = base_context[prefix] + suffix
            else:
                expanded_context[key] = value
        else:
            expanded_context[key] = value
    return expanded_context

def replace_keys_recursive(data, old_key, new_key):
    if isinstance(data, dict):
        for key in list(data.keys()):  # Create a copy of keys to avoid RuntimeError
            if key == old_key:
                key = new_key
                data[new_key] = data.pop(old_key)
            replace_keys_recursive(data[key], old_key, new_key)
    elif isinstance(data, list):
        for item in data:
            replace_keys_recursive(item, old_key, new_key)

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
parsed_url = urlparse.parse_qs(url.query)
params = {key:value[0] for key, value in parsed_url.items()}

molecule_json = json.loads(params["molecule_info"])

molecule_validator = set_up_validator("molecule.schema.json")
schema_context = molecule_validator.schema["@context"]
schema_context = expand_context(schema_context)
json_context = expand_context(molecule_json["@context"])

if "@context" in molecule_json:
    _ = molecule_json.pop("@context")
    
for key1, value1 in schema_context.items():
    for key2, value2 in json_context.items():
        if value1 == value2:
            replace_keys_recursive(molecule_json, key2, key1)
          
molecule_info_valid = False

try:
    molecule_validator.validate(instance = molecule_json)
    molecule_info_valid = True
    params["molecule_info"] = json.dumps(molecule_json)
    eln_widget = ElnImportWidget(path_to_root="../../", **params)
except ValidationError as e:
    message = e.schema["error_msg"] if "error_msg" in e.schema else e.message
    print(f'Invalid data: {message}')

In [ ]:
if molecule_info_valid:
    object_displayed = AiidaNodeViewWidget()
    open_in_app = OpenAiidaNodeInAppWidget(path_to_root="../../")

    _ = dlink((eln_widget, 'node'), (object_displayed, 'node'))
    _ = dlink((eln_widget, 'node'), (open_in_app, 'node'))

## Selected object:

In [ ]:
if molecule_info_valid:
    display(object_displayed)
    display(eln_widget)

## What's next?

In [ ]:
if molecule_info_valid:
    display(open_in_app)